In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import *
import datetime
import dateutil.relativedelta
import os
import sys
import argparse

spark = SparkSession.builder \
.master("local") \
.appName("CS in HT") \
.getOrCreate()

In [2]:
unp = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Unpaid_Installments_sample.csv")
unp.printSchema()
unp.registerTempTable('u')
unp.show(2)

root
 |-- Keydate_prompt: integer (nullable = true)
 |-- OIB: long (nullable = true)
 |-- CONTRACT_ACCOUNT: long (nullable = true)
 |-- Installment_Plan_No: integer (nullable = true)
 |-- Due_Date_YYYYMMDD: integer (nullable = true)
 |-- Due_Days_Inst_on_KeyDate: integer (nullable = true)
 |-- Amount: double (nullable = true)

+--------------+-----------+----------------+-------------------+-----------------+------------------------+------+
|Keydate_prompt|        OIB|CONTRACT_ACCOUNT|Installment_Plan_No|Due_Date_YYYYMMDD|Due_Days_Inst_on_KeyDate|Amount|
+--------------+-----------+----------------+-------------------+-----------------+------------------------+------+
|      20170731|49304755245|      4000087477|          210591030|         20170717|                      14| 651.0|
|      20170731|49304755245|      4000087477|          210591030|         20170617|                      44|651.87|
+--------------+-----------+----------------+-------------------+-----------------+--------

In [5]:
lock=spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Lock_by_DocNum_-_Complaints.csv")
lock.printSchema()
lock.registerTempTable('l')



root
 |-- Document_Number: long (nullable = true)
 |-- Lock_From_Date: integer (nullable = true)
 |-- Lock_To_Date: integer (nullable = true)



In [9]:
df=spark.sql("select * from u inner join l on u.Installment_Plan_No=l.Document_Number")
df.show()
print(df)

+--------------+-----------+----------------+-------------------+-----------------+------------------------+------+---------------+--------------+------------+
|Keydate_prompt|        OIB|CONTRACT_ACCOUNT|Installment_Plan_No|Due_Date_YYYYMMDD|Due_Days_Inst_on_KeyDate|Amount|Document_Number|Lock_From_Date|Lock_To_Date|
+--------------+-----------+----------------+-------------------+-----------------+------------------------+------+---------------+--------------+------------+
|      20170731|25536684637|      5005259746|          210417989|         20170616|                      45|  90.0|      210417989|      20181128|    20180125|
|      20170731|25536684637|      5005259746|          210417989|         20160615|                     411|  90.0|      210417989|      20181128|    20180125|
|      20170731|25536684637|      5005259746|          210417989|         20170515|                      77|  90.0|      210417989|      20181128|    20180125|
|      20170731|25536684637|      500525

In [8]:
collvl = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/CollectionLevel_by_DocNo_sample.csv")
collvl.printSchema()

collvl.registerTempTable('c')

root
 |-- Document_Number: long (nullable = true)
 |-- Due_Date: integer (nullable = true)
 |-- Dunning_Date: integer (nullable = true)
 |-- Collection_Level: integer (nullable = true)
 |-- Dunning_Amount: double (nullable = true)



In [14]:
join = df.join(collvl,"document_number","inner")
print(join)
join.registerTempTable('j')

df1=spark.sql("SELECT * FROM j WHERE j.Keydate_prompt not between j.Lock_From_Date AND j.Lock_To_Date")
df1.show()
print(df1)

DataFrame[Document_Number: bigint, Keydate_prompt: int, OIB: bigint, CONTRACT_ACCOUNT: bigint, Installment_Plan_No: int, Due_Date_YYYYMMDD: int, Due_Days_Inst_on_KeyDate: int, Amount: double, Lock_From_Date: int, Lock_To_Date: int, Due_Date: int, Dunning_Date: int, Collection_Level: int, Dunning_Amount: double]
+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+--------+------------+----------------+--------------+
|Document_Number|Keydate_prompt|OIB|CONTRACT_ACCOUNT|Installment_Plan_No|Due_Date_YYYYMMDD|Due_Days_Inst_on_KeyDate|Amount|Lock_From_Date|Lock_To_Date|Due_Date|Dunning_Date|Collection_Level|Dunning_Amount|
+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+--------+------------+----------------+--------------+
+---------------+--------------+---+----------------+----------------

In [15]:
df2 = df1.withColumn("Collection_Levels_Interval",f.expr("case when collection_level>=10 and collection_level<=19 then 1019\
                                        when collection_level>=20 and collection_level<=29 then 2029\
                                        when collection_level>=30 and collection_level<=39 then 3039\
                                        when collection_level>=40 and collection_level<=49 then 4049\
                                        when collection_level>=50 and collection_level<=79 then 5079\
                                        when collection_level>=80 and collection_level<=94 then 8094\
                                        when collection_level=95 then 95\
                                        when collection_level=0 then 'No_DUN' end"))
df2.show()
df2.registerTempTable('pd')

+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+--------+------------+----------------+--------------+--------------------------+
|Document_Number|Keydate_prompt|OIB|CONTRACT_ACCOUNT|Installment_Plan_No|Due_Date_YYYYMMDD|Due_Days_Inst_on_KeyDate|Amount|Lock_From_Date|Lock_To_Date|Due_Date|Dunning_Date|Collection_Level|Dunning_Amount|Collection_Levels_Interval|
+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+--------+------------+----------------+--------------+--------------------------+
+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+--------+------------+----------------+--------------+--------------------------+



In [18]:
df3=spark.sql("select Keydate_prompt,oib,amount,Collection_Levels_Interval \
from pd group by Keydate_prompt,oib,amount,Collection_Levels_Interval")
                                    
df3.show()

+--------------+---+------+--------------------------+
|Keydate_prompt|oib|amount|Collection_Levels_Interval|
+--------------+---+------+--------------------------+
+--------------+---+------+--------------------------+



In [19]:
dun_lvl = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/DunningLevel_by_DocNo_sample.csv")
dun_lvl.printSchema()



root
 |-- Document_Number: long (nullable = true)
 |-- Due_Date_YYYYMMDD: integer (nullable = true)
 |-- Dunning_Date_YYYYMMDD: integer (nullable = true)
 |-- Dunning_Level: integer (nullable = true)
 |-- Dunning_Amount: double (nullable = true)



In [20]:
join2 = df.join(dun_lvl,"document_number","inner")
print(join)
join2.registerTempTable('d')

df4=spark.sql("SELECT * FROM d WHERE Dunning_Date_YYYYMMDD <=Keydate_prompt")
df4.show()
print(df4)

DataFrame[Document_Number: bigint, Keydate_prompt: int, OIB: bigint, CONTRACT_ACCOUNT: bigint, Installment_Plan_No: int, Due_Date_YYYYMMDD: int, Due_Days_Inst_on_KeyDate: int, Amount: double, Lock_From_Date: int, Lock_To_Date: int, Due_Date: int, Dunning_Date: int, Collection_Level: int, Dunning_Amount: double]
+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+-----------------+---------------------+-------------+--------------+
|Document_Number|Keydate_prompt|OIB|CONTRACT_ACCOUNT|Installment_Plan_No|Due_Date_YYYYMMDD|Due_Days_Inst_on_KeyDate|Amount|Lock_From_Date|Lock_To_Date|Due_Date_YYYYMMDD|Dunning_Date_YYYYMMDD|Dunning_Level|Dunning_Amount|
+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+-----------------+---------------------+-------------+--------------+
+---------------+-------

In [23]:
df5 = df4.withColumn("Dunning_Level_Interval",f.expr("case when Dunning_Level>=1 and Dunning_Level<=9 then 19\
                                                           when Dunning_Level>=10 and Dunning_Level<=19 then 1019\
                                                           when Dunning_Level>=20 and Dunning_Level<=29 then 2029\
                                                           when Dunning_Level>=30 and Dunning_Level<=39 then 3039\
                                                           when Dunning_Level>=40 and Dunning_Level<=49 then 4049\
                                                           when Dunning_Level>=50 and Dunning_Level<=79 then 5079\
                                                           when Dunning_Level>=80 and Dunning_Level<=94 then 8094\
                                                           when Dunning_Level=95 then 95\
                                                           when Dunning_Level=0 then 'No_DUN' end"))
df5.show()
df5.registerTempTable('dl')
print(df5)

+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+-----------------+---------------------+-------------+--------------+----------------------+
|Document_Number|Keydate_prompt|OIB|CONTRACT_ACCOUNT|Installment_Plan_No|Due_Date_YYYYMMDD|Due_Days_Inst_on_KeyDate|Amount|Lock_From_Date|Lock_To_Date|Due_Date_YYYYMMDD|Dunning_Date_YYYYMMDD|Dunning_Level|Dunning_Amount|Dunning_Level_Interval|
+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+-----------------+---------------------+-------------+--------------+----------------------+
+---------------+--------------+---+----------------+-------------------+-----------------+------------------------+------+--------------+------------+-----------------+---------------------+-------------+--------------+----------------------+

DataFrame[Document_Numb

In [24]:
df6=spark.sql("select Keydate_prompt,oib,amount,Dunning_Level_Interval \
from dl group by Keydate_prompt,oib,amount,Dunning_Level_Interval")
                                    
df6.show()

+--------------+---+------+----------------------+
|Keydate_prompt|oib|amount|Dunning_Level_Interval|
+--------------+---+------+----------------------+
+--------------+---+------+----------------------+

